<a href="https://colab.research.google.com/github/oakthyago/DC12-A-Solution-for-Managing-Electrical-Contract-Demand-in-Brazil/blob/main/NLP_Introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
from google.colab import files
import os
import shutil

# Upload kaggle.json
uploaded = files.upload()

# Move kaggle.json to the .kaggle directory
os.makedirs('/root/.kaggle', exist_ok=True)
shutil.move('kaggle.json', '/root/.kaggle/kaggle.json')

# Set permissions
!chmod 600 /root/.kaggle/kaggle.json

# Verify setup
!kaggle datasets list


Saving kaggle.json to kaggle.json
ref                                                          title                                              size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
haseebindata/student-performance-predictions                 Student Performance Predictions                     9KB  2024-08-17 06:57:57           8653        192  0.9411765        
lainguyn123/student-performance-factors                      Student Performance Factors                        94KB  2024-09-02 10:53:57           2298         53  1.0              
stefanydeoliveira/summer-olympics-medals-1896-2024           Summer Olympics Medals (1896-2024)                  4MB  2024-08-29 23:52:10           1693         36  1.0              
abdullah0a/human-age-prediction-synthetic-dataset  

In [24]:
# prompt: get this data set from kaggle "luisfredgs/imdb-ptbr"

!kaggle datasets download -d luisfredgs/imdb-ptbr
# prompt: unzip

!unzip imdb-ptbr.zip


Dataset URL: https://www.kaggle.com/datasets/luisfredgs/imdb-ptbr
License(s): unknown
imdb-ptbr.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  imdb-ptbr.zip
  inflating: imdb-reviews-pt-br.csv  


In [27]:
# prompt: df = imdb-ptbr

import pandas as pd

df = pd.read_csv('imdb-reviews-pt-br.csv')


In [28]:
df.head()

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg
